# Preprocessing and geocoding 

In [ ]:
import re
import requests
import pandas as pd 

In [ ]:
#Importation dans le notebook du fichier voulu 
fichier = "constructions_non_resid"
df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/Liste-des-autorisations-durbanisme-creant-des-locaux-non-residentiels.2025-09.csv", sep=";")

/tmp/ipykernel_11704/1571034802.py:3: DtypeWarning: Columns (0,2,4,7,11,17,19,20,21,23,29,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/Liste-des-autorisations-durbanisme-creant-des-locaux-non-residentiels.2025-09.csv", sep=";")


In [ ]:
df.columns

Index(['Code de la région du lieu des travaux - Code de la zone',
       'Code de la région du lieu des travaux - Libellé de la zone',
       'Code du département du lieu des travaux - Code de la zone',
       'Code du département du lieu des travaux - Libellé de la zone',
       'Code de la commune du lieu des travaux',
       'Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)',
       'Numéro d'enregistrement de la DAU', 'Etat d'avancement du projet',
       'Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale',
       'Date réelle d'ouverture de chantier',
       'Date réelle d'achèvement des travaux', 'Année de dépôt de la DAU',
       'Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)',
       'Date (mois) de prise en compte (DPC) de l'autorisation initiale',
       'Date (mois) de prise en compte (DPC) de l'ouverture du chantier',
       'Date (mois) (DPC) de derniè

Objectif à présent : représenter sur une carte les différents points de construction
  On peut récupérer les géométries des adresses avec cette API
  https://adresse.data.gouv.fr/outils/api-doc/adresse
  https://data.geopf.fr/geocodage/openapi 

In [5]:
elements_adresse = ['Numéro de voie du terrain',
       'Type de voie du terrain', 'Libellé de la voie du terrain',
       'Lieu-dit du terrain', 'Localité du terrain'] 
df_adresses = df.loc[:,elements_adresse]
df['Adresse_complete'] = df_adresses.apply(lambda row : " ".join(row.dropna().astype(str)), axis=1)


In [ ]:
df.head()

,Code de la région du lieu des travaux - Code de la zone,Code de la région du lieu des travaux - Libellé de la zone,Code du département du lieu des travaux - Code de la zone,Code du département du lieu des travaux - Libellé de la zone,Code de la commune du lieu des travaux,"Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)",Numéro d'enregistrement de la DAU,Etat d'avancement du projet,Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale,Date réelle d'ouverture de chantier,...,Surface de plancher de la destination Entrepôt issue d'une transformation,Surface de plancher de la destination Entrepôt démolie,Surface de plancher de la destination Entrepôt transformée,Surface de plancher de la destination Services publics ou d'intérêt collectif' existante avant travaux,Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite,Surface de plancher de la destination Services publics ou d'intérêt collectif issue d'une transformation,Surface de plancher de la destination Services publics ou d'intérêt collectif démolie,Surface de plancher de la destination Services publics ou d'intérêt collectif transformée,Type de service public ou d'intérêt collectif,Adresse_complete
0,REG_CODE,REG_LIBELLE,DEP_CODE,DEP_LIBELLE,COMM,TYPE_DAU,NUM_DAU,ETAT_DAU,DATE_REELLE_AUTORISATION,DATE_REELLE_DOC,...,SURF_ENT_ISSUE_TRANSFO,SURF_ENT_DEMOLIE,SURF_ENT_TRANSFORMEE,SURF_PUB_AVANT,SURF_PUB_CREEE,SURF_PUB_ISSUE_TRANSFO,SURF_PUB_DEMOLIE,SURF_PUB_TRANSFORMEE,TYPE_SERVICE_PUBLIC,ADR_NUM_TER ADR_TYPEVOIE_TER ADR_LIBVOIE_TER A...
1,82,Rhône-Alpes,01,Ain,01001,PC,00100117V0002,5,2017-05-05,2017-06-01,...,0,0,0,0,0,0,0,0,7,ABERGEMENT CLEMENCIAT
2,82,Rhône-Alpes,01,Ain,01001,PC,00100119V0005,2,2019-08-29,NaN,...,0,0,0,0,0,0,0,0,7,203 Rue des cinq Chemins ABERGEMENT CLEMENCIAT
3,82,Rhône-Alpes,01,Ain,01001,PC,00100120V0003,5,2020-05-25,2024-07-08,...,0,0,0,0,781,0,0,0,6,172 Route du Chateau ABERGEMENT CLEMENCIAT
4,82,Rhône-Alpes,01,Ain,01001,DP,00100119V0011,2,2019-07-22,NaN,...,0,0,0,0,0,0,0,0,7,Sur Brody ABERGEMENT CLEMENCIAT


## Geocding of the whole CSV directly through the API (the entire file but error 413)

In [ ]:
df.to_csv(f'{fichier}_prétraité.csv', encoding='utf-8')

In [11]:

params = {
    'columns': "Adresse_complete",
}

files = {
    'data': open(f'{fichier}_prétraité.csv', 'rb')
}


response = requests.post(
    'https://api-adresse.data.gouv.fr/search/csv/',
    files=files,
    data=params  
)


In [ ]:
print("Status code :", response.status_code)

OK ? False
Status code : 413


In [16]:
if response.ok:
    with open(f'{fichier}_geocode.csv', 'wb') as f:
        f.write(response.content)

df = pd.read_csv(f'{fichier}_geocode.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'constructions_non_résid_geocode.csv'

## Geocoding par chunk

In [ ]:
dfs_results = [] 
chunksize = 50000
for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):
    print(f"Chunk {i + 1}")

    csv_data = chunk.to_csv(index=False)

    files = {
        "data": ("chunk.csv", csv_data, "text/csv")
    }

    response = requests.post(
        'https://api-adresse.data.gouv.fr/search/csv/',
        files=files,
        data=params,
        timeout=60
    )

    print(response.status_code)

    if not response.ok:
        raise RuntimeError(
            f"Erreur sur le chunk {i + 1} "
            f"({response.status_code})"
        )

    # lire la réponse CSV retournée par l’API
    df_chunk_geocode = pd.read_csv(
        pd.io.common.BytesIO(response.content)
    )

    dfs_results.append(df_chunk_geocode)
df_final = pd.concat(dfs_results, ignore_index=True)

NameError: name 'pd' is not defined

In [ ]:
df_final.to_csv(f"{fichier}_geocoded.csv", index=False, encoding="utf-8")
print(f"File created")

## Line-per-line geocoding 

In [ ]:
#Boucle qui permet d'obtenir manuellement les géométries
import requests
def geocodage(adresse) :
        try:
                url = f"https://api-adresse.data.gouv.fr/search/?q={adresse.replace(' ', '+')}"
                data = requests.get(url, timeout=1).json()
        
                if data.get('features'):
                        return data['features'][0]['geometry']['coordinates']
        except:
                pass 


adresses["Géométries"] = adresses["Adresse_complete"].apply(geocodage)

In [ ]:
adresses.head(10)

,Numéro de voie du terrain,Type de voie du terrain,Libellé de la voie du terrain,Lieu-dit du terrain,Localité du terrain,Adresse_complete,Géométries
35693,NaN,NaN,Avenue Jacques Dupin,NaN,PRIVAS,Avenue Jacques Dupin PRIVAS,"[4.611476, 44.722614]"
369554,NaN,Ld,L oiseliere,NaN,La Rouxiere,Ld L oiseliere La Rouxiere,"[-1.089582, 47.44852]"
323031,2,NaN,rue en villeneuve,NaN,Baume-les-Messieurs,2 rue en villeneuve Baume-les-Messieurs,"[5.647772, 46.706858]"
411579,NaN,rue,du Souvenir,NaN,TREMENTINES,rue du Souvenir TREMENTINES,"[-0.787733, 47.119167]"
7462,5,NaN,Route de Meillonnas,TREFFORT,VAL-REVERMONT,5 Route de Meillonnas TREFFORT VAL-REVERMONT,"[5.367165, 46.269018]"
531723,NaN,NaN,MITIGNA OYHANE,NaN,BEYRIE SUR JOYEUSE,MITIGNA OYHANE BEYRIE SUR JOYEUSE,"[-1.07557, 43.303025]"
436936,NaN,NaN,NaN,La Goupillais,La Rouaudiere,La Goupillais La Rouaudiere,"[-1.197518, 47.845472]"
103488,NaN,NaN,le Bourg,NaN,Brie-sous-Chalais,le Bourg Brie-sous-Chalais,"[0.007469, 45.315502]"
335661,622,AV,DES GRANDS LACS,NaN,SANGUINET,622 AV DES GRANDS LACS SANGUINET,"[-1.081388, 44.479231]"
738179,NaN,NaN,chemin d exploitation n 1,L etang des Sanguinots,Courgis,chemin d exploitation n 1 L etang des Sanguin...,"[3.75669, 47.776651]"


#FONCTION POUR TRANSFORMER EN LIGNE UN DICTIONNAIRE 
    json_normalize(
        data=req['features'],  # Quelle liste aplatir
        record_path=['properties'],      # Chemin vers l'objet à "déplier"
        meta=['type'],
        sep='_',
        errors="ignore"                         # Séparateur pour noms de colonnes imbriqués
    )